In [ ]:
import chess.pgn
import chess.svg
import chess
import os
from flask import Flask, render_template, request
from natsort import natsorted
import shutil
import ast
import matplotlib.pyplot as plt
import os

# Connect to MongoDB
from pymongo import MongoClient
client = MongoClient('localhost', 27017) ## or MongoClient("localhost:27")
mongodb = client.chessDB
gameCollection = mongodb.gameCollection
playerCollection = mongodb.playerCollection

In [ ]:
def getAllGames(name):
    query = { "$or": [
    {"headers.White": name},
    {"headers.Black": name}
    ]}
    documents = gameCollection.find(query)
    
    return documents

In [ ]:
def getRatingPlot(player):
    x_values = player['dates'] 
    y_values = player['ratings'] 

    # Plot the graph
    plt.plot(x_values, y_values)
    plt.xlabel('Date')
    plt.ylabel('Rating')
    plt.title('Plot of rating vs date for {}'.format(player['name']))
    # save the plot to a file in the output directory
    filename = 'static/output/rating.png'
    plt.savefig(filename)

    # close the plot
    plt.close()

In [ ]:
def getStats(name):
    query = {"name": name}
    documents = playerCollection.find(query)

    shutil.rmtree('static/output')

    # create the output directory if it doesn't already exist
    if not os.path.exists('static/output'):
        os.makedirs('static/output')

    for doc in documents:
        getRatingPlot(doc)
        return doc

In [ ]:
def getGameplay(game):
    pgn = ''
    moves = ast.literal_eval(game)
    
    move_number = 1
    for i in range(1, len(moves), 2):
        pgn += str(move_number) + '. ' + moves[i-1] + ' ' + moves[i] + ' '
        move_number += 1

    pgn += '*'

    with open("static/game.pgn", "w") as file:
        file.write(pgn)

    folder_path = 'static/gameImages/'

    # Use shutil.rmtree() to delete everything from the folder
    if os.path.exists(folder_path):
        shutil.rmtree(folder_path)

    # Recreate the folder if needed
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)


    pgn_file = open("static/game.pgn")
    pgn_game = chess.pgn.read_game(pgn_file)
    board = pgn_game.board()

    board_image = chess.svg.board(board=board)
    with open("static/gameImages/board0.svg", "w") as svg_file:
        svg_file.write(board_image)

    moveNumber = 1
    for move in pgn_game.mainline_moves():
        board.push(move)

        board_image = chess.svg.board(board=board)

        with open("static/gameImages/board{}.svg".format(moveNumber), "w") as svg_file:
            svg_file.write(board_image)
        moveNumber += 1


In [12]:
app = Flask("ChessGame", template_folder=r'templates')

@app.route('/')
def index():
    return render_template("Home.html")

@app.route('/gameSection', methods=['GET', 'POST'])
def getPlayerNames():
    options = []
    for document in playerCollection.find():
        options.append(document['name'])
    if request.method == 'POST':
        selectedPlayer = request.form.get('player_1_name')
        games = getAllGames(selectedPlayer)
        return render_template('ViewMatches.html', options=options, games=games, selectedPlayer=selectedPlayer)
    return render_template('ViewMatches.html', options=options)

@app.route('/<game>', methods=['GET'])
def getGameDetails(game):
    headers = request.args.get('headers')
    headers = ast.literal_eval(headers)
    getGameplay(game)
    image_names = []
    folder_path = "static/gameImages/"  # replace with the actual path to your folder
    files = os.listdir(folder_path)
    for file in files:
        if os.path.isfile(os.path.join(folder_path, file)):
            image_names.append('gameImages/'+file)
    image_names = natsorted(image_names)
    return render_template('GameReplay.html', image_names=image_names, headers=headers)


@app.route('/playerSection', methods=['GET', 'POST'])
def getPlayerStats():
    options = []
    for document in playerCollection.find():
        options.append(document['name'])
    if request.method == 'POST':
        selectedPlayer = request.form.get('player_1_name')
        stats = getStats(selectedPlayer)
        return render_template('ViewPlayers.html', options=options, stats=stats, selectedPlayer=selectedPlayer)
    return render_template('ViewPlayers.html', options=options)

@app.route('/contact')
def contact():
    return render_template('Contact.html')
    
app.run(host='localhost', port=5001)
